In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Q, Search
from datetime import datetime

client = Elasticsearch(["http://127.0.0.1:9200"])

INDEX = "openfv"

# COMMITS (MERGES) TO MASTER/STABLE BRANCHES BY REPO

In [172]:
q = Q('bool', 
      should=[Q('match', message_analyzed='stable'), Q('match', message_analyzed='master')])
s = Search(using=client, index=INDEX).query(q).filter('range', files={'lt': 1})

s.aggs.bucket('by_repo', 'terms', field='repo_name', size=10).metric('commits', 'cardinality', field='hash')
result = s.execute()

buckets = result.to_dict()["aggregations"]['by_repo']['buckets']

commits_by_repo = [{'commits': i['commits']['value'], 'repo': i['key']} for i in buckets]

merges_to_master = pd.DataFrame(commits_by_repo)

first_ten_repos = list(merges_to_master['repo'])

merges_to_master

,commits,repo
0,73,git://git.opnfv.org/fuel
1,54,git://git.opnfv.org/apex
2,41,git://git.opnfv.org/compass4nfv
3,30,git://git.opnfv.org/armband
4,30,git://git.opnfv.org/functest
5,27,git://git.opnfv.org/releng
6,15,git://git.opnfv.org/yardstick
7,14,git://git.opnfv.org/copper
8,12,git://git.opnfv.org/qtip
9,11,git://git.opnfv.org/genesis


Como podemos ver, el ranking de repositorios en función de los merge a ramas master o stable cambia bastante. 

Consideramos que este ranking puede ser interesante ya que cuando se trabaja con ramas master o stable se está trabajando con versiones de producción, ya bien sea para pasar una nueva funcionalidad a producción, crear nuevas funcionalidades en otras ramas o arreglar bugs.

# REPO THAT MERGES FASTER TO MASTER/STABLE BRANCH

In [175]:
q = Q('bool', should=[Q('match', message_analyzed='stable'), Q('match', message_analyzed='master')])
s = Search(using=client, index=INDEX).query(q).filter('range', files={'lt': 1})
count = s.count()
result = s[0:count].execute()

date_and_repo_name = [{'date': np.datetime64(i['_source']['utc_commit']), 'repo': i['_source']['repo_name']} 
                       for i in result["hits"]["hits"]]

df = pd.DataFrame(date_and_repo_name)

df = df[df['repo'].isin(first_ten_repos)]

df['diff'] = (df.sort_values(by='date')
 .groupby('repo')
 .diff()
) 

df['diff'] = df['diff'] / np.timedelta64(1, 'D')

print('Average day diff by repo')
print(df.groupby('repo')['diff'].mean().sort_values(inplace=False))

print('\n Median day diff by repo')
print(df.groupby('repo')['diff'].median().sort_values(inplace=False))

Average day diff by repo
repo
git://git.opnfv.org/fuel            5.777752
git://git.opnfv.org/apex            7.722825
git://git.opnfv.org/compass4nfv    10.765165
git://git.opnfv.org/armband        11.015622
git://git.opnfv.org/copper         12.905901
git://git.opnfv.org/functest       14.284831
git://git.opnfv.org/genesis        19.286286
git://git.opnfv.org/releng         25.009750
git://git.opnfv.org/yardstick      27.967160
git://git.opnfv.org/qtip           39.278572
Name: diff, dtype: float64

 Median day diff by repo
repo
git://git.opnfv.org/qtip            0.007211
git://git.opnfv.org/compass4nfv     0.069196
git://git.opnfv.org/copper          0.187986
git://git.opnfv.org/fuel            0.346453
git://git.opnfv.org/apex            0.770671
git://git.opnfv.org/armband         2.161215
git://git.opnfv.org/yardstick       2.421655
git://git.opnfv.org/functest        2.834387
git://git.opnfv.org/genesis         3.821493
git://git.opnfv.org/releng         16.340521
Name: diff, 

Aquí se puede ver la diferencia de días media (arriba) y mediana (abajo) para cada repositorio. Sólo se han tenido en cuenta los 10 primeros repositorios, que son los que se muestran en la tabla anterior a estas.

En la tabla de las medias, se aprecia que no cambia mucho con respecto a la del número de merges de master/stable, pero si nos fijamos en la tabla de las medianas hay cambios en las primeras posiciones. 

El primer repositorio que aparece en la tabla de las medianas es el segundo que menos commits de este tipo tiene, por lo que no es muy significativa esa diferencia mediana. El segundo (compass4nfv) es el tercero que más commits de este tipo tiene así que en este caso la diferencia mediana sí puede ser indicativa. Los casos de fuel y apex también son interesantes dado que son los dos que más commits de merge master/stable tienen y también se encuentran entre las primeras posiciones de este ranking.

# COMMITS/AUTHORS BY REPO

In [28]:
#q_commits = Q('bool', should=[Q('match', message_analyzed='stable'), Q('match', message_analyzed='master')])
#s_commits = Search(using=client, index=INDEX).query(q).filter('range', files={'lt': 1})

#(s_commits.aggs.bucket('by_repo', 'terms', field='repo_name')
# .metric('commits', 'cardinality', field='hash')
#)

#count = s_commits.count()
#result_commits = s_commits[0:count].execute()

#docs_commits = result_commits.to_dict()["aggregations"]["by_repo"]["buckets"]

#s_authors = Search(using=client, index=INDEX)

#(s_authors.aggs.bucket('by_repo', 'terms', field='repo_name')
# .metric('authors', 'cardinality', field='author_id')
#)

#result_authors = s_authors.execute()

#docs_authors = result_authors.to_dict()["aggregations"]["by_repo"]["buckets"]

#authors_by_repo_dict = {}

#for d in docs_authors:
#    authors_by_repo_dict[d['key']] = d['authors']['value']
    

#docs_ratio = [{'repo': d['key'], 
#             'commits': d['commits']['value'], 
#             'devs': authors_by_repo_dict[d['key']], 
#             'ratio': d['commits']['value']/authors_by_repo_dict[d['key']]} 
#            for d in docs_commits]

#pd.DataFrame(docs_ratio)

In [40]:
# En Kibana fue necesario ir a la pestaña Dev Tools y hacer esto: 
#PUT _settings
#{ "index" : { "max_result_window" : 500000 } }

s = Search(using=client, index=INDEX).filter('range', files={'gt':0})

(s.aggs.bucket('by_repo', 'terms', field='repo_name', size=10)
 .metric('commits', 'cardinality', field='hash')
 .metric('devs', 'cardinality', field='author_id')
)

count = s.count()
result = s[0:count].execute()

docs = result.to_dict()["aggregations"]["by_repo"]["buckets"]

docs = [{'repo': d['key'], 
        'commits': d['commits']['value'], 
        'devs': d['devs']['value'], 
        'ratio': d['commits']['value']/d['devs']['value']} 
       for d in docs]

pd.DataFrame(docs).sort_values(by='ratio', ascending=False)

,commits,devs,ratio,repo
7,722,21,34.380952,git://git.opnfv.org/joid
4,847,38,22.289474,git://git.opnfv.org/apex
1,1800,88,20.454545,git://git.opnfv.org/functest
0,2472,128,19.312500,git://git.opnfv.org/releng
6,743,40,18.575000,git://git.opnfv.org/compass4nfv
2,968,54,17.925926,git://git.opnfv.org/armband
9,456,28,16.285714,git://git.opnfv.org/qtip
5,776,53,14.641509,git://git.opnfv.org/fuel
3,908,66,13.757576,git://git.opnfv.org/yardstick
8,476,39,12.205128,git://git.opnfv.org/vswitchperf


En cuanto a commits por autor de cada repositorio, se observa que "apex" es el que segundo mejor ratio tiene. "compass4nfv" se encuentra ligeramente por encima del valor de ratio mediano. En esta tabla sale peor parada "fuel" que está muy abajo.

Con los tres últimos apartados uno se puede hacer una idea de qué repositorio tiene los equipos de desarrolladores que más rápido trabajan (sacan versiones estables, arreglan fallos sobre versiones estables, entre otros) sobre master/stable y además, en general, cuántos commits por desarrollador hay por proyecto.

Las empresas que mejor posicionadas están conforme los tres últimos apartados son "apex", "fuel" y "compass4nfv". Dado que "fuel", como hemos comentado, aparece muy baja en esta última tabla, podríamos decir que los equipos de desarrollo de "apex" y "compass4nfv" son los más interesantes en cuanto a contribuciones y trabajo sobre master/stable.

Si se mira la serie temporal de commits de la visualización de Kibana, "apex" parece tener un número de commits mensual más alto que "compass4nfv", por lo que si se quisiera invertir por un proyecto quizás invertiríamos por "apex".